In [36]:
from lxml import html, etree
import urllib2
import requests
import HTMLParser

url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://18.fsin.su/uchrezdeniya/msch-18/')
tree = html.document_fromstring(page.content)
aa = tree.xpath("//div[@class='l-content-holder']")[0]
print HTMLParser.HTMLParser().unescape(html.tostring(aa))

<div class="l-content-holder">
                                <div class="b-content margin">
                                    <div class="bb-title"><h1 class="h1">Федеральное казеное учреждение здравоохранения "Медико-Санитарная Часть № 18 ФСИН России"</h1></div>
                                    <div class="separator"></div>
                         
<p class="MsoNormal"><font size="2" face="Arial, sans-serif" color="#333333"><b>Почтовый адрес: </b></font></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">426004, г. Ижевск, ул. Коммунаров, 216</span></p>
 
<p class="MsoNormal"><b style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">Контактные телефоны: </b></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">Приемная: 8 (3412) 631-937</span></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Aria

In [ ]:
import HTMLParser
url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://18.fsin.su/uchrezdeniya/msch-18/')
HTMLParser.HTMLParser().unescape(page.content)

In [ ]:
from BeautifulSoup import BeautifulStoneSoup

In [ ]:
import SimpleHTTPServer
import SocketServer

class MyRequestHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
    def do_GET(self):
        #if self.path == '/':
        self.path = '1.html'
        return SimpleHTTPServer.SimpleHTTPRequestHandler.do_GET(self)

Handler = MyRequestHandler
server = SocketServer.TCPServer(('', 8000), Handler)

server.serve_forever()

In [4]:
import sqlite3 as lite
import sys

try:
    con = lite.connect('test.db')

    cur = con.cursor()  

    cur.executescript("""
        CREATE TABLE IF NOT EXISTS Cars(Id INT, Name TEXT, Price INT);
        INSERT INTO Cars VALUES(1,'Audi',52642);
        INSERT INTO Cars VALUES(2,'Mercedes',57127);
        INSERT INTO Cars VALUES(3,'Skoda',9000);
        INSERT INTO Cars VALUES(4,'VolvoАААббб',29000);
        INSERT INTO Cars VALUES(5,'Bentley',350000);
        INSERT INTO Cars VALUES(6,'Citroen',21000);
        INSERT INTO Cars VALUES(7,'Hummer',41400);
        INSERT INTO Cars VALUES(8,'Volkswagen',21600);
        """)

    con.commit()
    cur.execute("SELECT * FROM Cars")

    rows = cur.fetchall()

    for row in rows:
        print row[1]
    
except lite.Error, e:
    
    if con:
        con.rollback()
        
    print "Error %s:" % e.args[0]
    sys.exit(1)
    
finally:
    
    if con:
        con.close() 

Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen
Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen


In [18]:
import flask
import simplejson as json
import sqlite3 as lite
import requests
from lxml import html
import HTMLParser

class WebApplication:
    def __init__(self):
        self.app = flask.Flask(__name__, static_url_path = '')

        @self.app.route('/js/<path:path>')
        def send_js(path):
            return flask.send_from_directory('js', path)

        @self.app.route('/', methods = ['GET', 'POST'])
        def login():
            if flask.request.method == 'POST':
                print flask.request.form['name']
                return 'OK'
            else:
                print 234
                return self.app.send_static_file('index.html')
    
    def run(self):
        self.app.run()

class FsinParser:
    def __init__(self):
        self.json_name = '1.json'
        with open(self.json_name, 'r') as f:
            self.json = json.load(f)
            f.close()
        self.db = lite.connect(self.json['db_name'])
        try:
            self.db.cursor().executescript("""CREATE TABLE IF NOT EXISTS \
                                           Pages(Id INT, Info TEXT, Datetime STRING);""")
        except:
            if self.db:
                self.db.rollback()
            print '__init__: SQL Error'
        
    
    def get_info_from_page(self, index):
        url = self.json['pages'][str(index)]
        page = requests.get(url)
        tree = html.document_fromstring(page.content)
        info = tree.xpath(self.json['xpath_request'])[0]
        return HTMLParser.HTMLParser().unescape(html.tostring(info)).encode('utf-8')
    
    def put_info_to_db(self, index, data):
        try:
            self.db.cursor().executescript("""INSERT INTO Cars VALUES(2,'Mercedes',57127);""")
        except:
            'put_info_to_db: SQL Error'
        
    def finish(self):
        if (self.db):
            self.db.close()
        
    

if __name__ == '__main__':
    # app = WebApplication()
    # app.run()
    parser = FsinParser()
    with open('out1.txt', 'w') as f:
        f.write(parser.get_info_from_page(0))
        f.close()

In [82]:
import difflib
from lxml.html.diff import htmldiff
html_diff = difflib.HtmlDiff()

url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://18.fsin.su/uchrezdeniya/msch-18/')
tree = html.document_fromstring(page.content)
aa = tree.xpath("//div[@class='l-content-holder']")[0]
data =  HTMLParser.HTMLParser().unescape(html.tostring(aa))
data1 = HTMLParser.HTMLParser().unescape(html.tostring(aa))

data1 = data1.replace('ФСИН'.decode('utf-8'), 'jopa')
#print ''.join(difflib.context_diff(data, data1))

diff = HTMLParser.HTMLParser().unescape(htmldiff(data, data1).encode('utf-8'))
diff = diff.replace('<del>', '<del style="color:red">')
diff = diff.replace('<ins>', '<ins style="color:green">')
with open('table.html', 'w') as f:
    f.write(HTMLParser.HTMLParser().unescape(diff))